In [ ]:
import numpy as np
import cv2 as cv
import os

import sys

sys.path.append("..")

import json
import matplotlib.pyplot as plt

# import apriltag
from scipy.spatial.transform import Rotation

import lac.params as params
from lac.utils.frames import (
    apply_transform,
    invert_transform_mat,
    get_cam_pose_rover,
    OPENCV_TO_CAMERA_PASSIVE,
)
from lac.perception.vision import FiducialLocalizer
from lac.utils.visualization import overlay_tag_detections

%load_ext autoreload
%autoreload 2

# AprilTag detection


In [ ]:
# data_path = os.path.expanduser("../../output/lander_fiducials_right_light")
data_path = os.path.expanduser("../../output/nav_agent_preset_0_seed_4")
json_data = json.load(open(f"{data_path}/data_log.json"))
lander_pose = np.array(json_data["lander_pose_world"])
poses = np.array([frame["pose"] for frame in json_data["frames"]])

In [ ]:
camera_config = json_data["cameras"]

fid_localizer = FiducialLocalizer(camera_config)

In [ ]:
i = 60
CAM_NAME = "FrontLeft"
img = cv.imread(os.path.join(data_path, CAM_NAME, f"{i}.png"), cv.IMREAD_GRAYSCALE)

detections = fid_localizer.detect(img)
overlay = overlay_tag_detections(img, detections)
plt.imshow(overlay)

In [ ]:
i = 2000
CAM_NAME = "Right"
img = cv.imread(os.path.join(data_path, CAM_NAME, f"{i}.png"), cv.IMREAD_GRAYSCALE)

detections = fid_localizer.detect(img)
overlay = overlay_tag_detections(img, detections)
plt.imshow(overlay)

# Generate pose measurements


In [ ]:
fid_localizer.estimate_rover_pose(img, CAM_NAME, lander_pose)

In [ ]:
rover_pose = np.array(json_data["frames"][i]["pose"])
rover_pose

# Analyze Outliers


In [ ]:
# TODO: :disguised_face: